# data set 
Foi pego no uci, convertido para csv com auxilio de script


In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from statistics import mean 


In [2]:
headers = list(pd.read_csv("EGG_EyeStateHeaders.csv") )

In [3]:
data = pd.read_csv("EEG_EyeState.csv", names=headers, index_col=False, skiprows = [0]) 
# Preview the first 5 lines of the loaded data 
data.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, data.eyeDetection, test_size=0.3)                                                    
X_train = X_train.drop("eyeDetection", axis=1)
X_test = X_test.drop("eyeDetection", axis=1)

In [5]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)

           AF3       F7       F3      FC5       T7       P7       O1       O2  \
5158   4335.90  4022.05  4275.38  4120.51  4345.13  4624.10  4070.26  4602.05   
1203   4299.49  4038.46  4261.54  4141.03  4337.44  4584.10  4083.08  4598.46   
11212  4289.74  3972.82  4267.69  4105.13  4348.72  4634.87  4087.69  4631.28   
2839   4273.85  3976.41  4249.23  4101.54  4323.59  4608.72  4086.15  4608.72   
9726   4261.54  3966.67  4249.23  4089.23  4318.97  4586.15  4040.00  4600.00   
...        ...      ...      ...      ...      ...      ...      ...      ...   
4438   4268.72  3958.97  4283.08  4126.67  4335.38  4621.03  4070.26  4612.82   
419    4305.64  3975.38  4263.59  4090.26  4326.15  4621.03  4106.67  4641.03   
11873  4411.28  4086.15  4340.00  4208.72  4404.62  4669.23  4132.31  4678.46   
3350   4415.38  4048.21  4281.54  4158.46  4347.69  4600.00  4069.23  4585.13   
2784   4271.79  3982.56  4251.79  4116.41  4331.79  4617.95  4087.69  4625.13   

            P8       T8    

In [6]:
weights = ['uniform', 'distance'] # falta um terceiro??
distances = ['canberra', 'chebyshev', 'correlation', 'euclidean','cosine' ] #'mahalanobis'
V = np.cov(X_train)

In [7]:
def cross_validation(knn_k, distance, weight_function, kfold = 5):
    #Create KNN Classifier
    if distance == 'mahalanobis':
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
    else:
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)
        
    kf = KFold(n_splits= kfold)
    accs = []
    for train_index, test_index in kf.split(X_train):
        fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
        fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
        #Train the model using the training sets
        knn.fit(fold_x_train, fold_y_train)

        #Predict the response for test dataset
        fold_y_pred = knn.predict(fold_x_test)

        acc = metrics.accuracy_score(fold_y_test, fold_y_pred)
        # Model Accuracy, how often is the classifier correct?
        print("Accuracy:", acc)
        accs.append(acc)
    
    return [knn_k, distance, weight_function, mean(accs)]

In [8]:
def test(knn_k, distance, weight_function):
    #Create KNN Classifier
    if distance == 'mahalanobis':
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
    else:
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

    #Train the model using the training sets
    knn.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = knn.predict(X_test)


    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [9]:
def select_bests_params(results):
    df_results = pd.DataFrame(results)
    return df_results.sort_values([3],  ascending=[0])

In [10]:
results = []
for weight_function in weights:
    for knn_k in range(1,16,2):
        for distance in distances:
            print("Função: {} - distancia: {} - K: {}".format(weight_function,distance, knn_k))
            results.append(cross_validation(knn_k, distance, weight_function, kfold = 5))

ranking = select_bests_params(results)
print(ranking)

Função: uniform - distancia: canberra - K: 1
Accuracy: 0.553004726536
Accuracy: 0.567143830948
Accuracy: 0.53825136612
Accuracy: 0.482415394824
Accuracy: 0.383768913343
Função: uniform - distancia: chebyshev - K: 1
Accuracy: 0.540175557056
Accuracy: 0.564417177914
Accuracy: 0.498633879781
Accuracy: 0.469143994691
Accuracy: 0.392022008253
Função: uniform - distancia: correlation - K: 1
Accuracy: 0.577312626604
Accuracy: 0.699386503067
Accuracy: 0.502049180328
Accuracy: 0.469807564698
Accuracy: 0.401650618982
Função: uniform - distancia: euclidean - K: 1
Accuracy: 0.538149898717
Accuracy: 0.553510565781
Accuracy: 0.515710382514
Accuracy: 0.459854014599
Accuracy: 0.366574965612
Função: uniform - distancia: cosine - K: 1
Accuracy: 0.501012829169
Accuracy: 0.644853442399
Accuracy: 0.480191256831
Accuracy: 0.510948905109
Accuracy: 0.399587345254
Função: uniform - distancia: canberra - K: 3
Accuracy: 0.548278190412
Accuracy: 0.584867075665
Accuracy: 0.551229508197
Accuracy: 0.465826144658
Acc

Accuracy: 0.591683708248
Accuracy: 0.53825136612
Accuracy: 0.474452554745
Accuracy: 0.359009628611
Função: distance - distancia: cosine - K: 3
Accuracy: 0.512491559757
Accuracy: 0.6530334015
Accuracy: 0.494535519126
Accuracy: 0.504313205043
Accuracy: 0.398899587345
Função: distance - distancia: canberra - K: 5
Accuracy: 0.551654287643
Accuracy: 0.592365371506
Accuracy: 0.564207650273
Accuracy: 0.469143994691
Accuracy: 0.366574965612
Função: distance - distancia: chebyshev - K: 5
Accuracy: 0.538149898717
Accuracy: 0.588957055215
Accuracy: 0.512978142077
Accuracy: 0.434638354346
Accuracy: 0.389270976616
Função: distance - distancia: correlation - K: 5
Accuracy: 0.575286968265
Accuracy: 0.700749829584
Accuracy: 0.522540983607
Accuracy: 0.497677504977
Accuracy: 0.392022008253
Função: distance - distancia: euclidean - K: 5
Accuracy: 0.550303848751
Accuracy: 0.597137014315
Accuracy: 0.545081967213
Accuracy: 0.467816854678
Accuracy: 0.373452544704
Função: distance - distancia: cosine - K: 5
A

In [11]:
best_knn_k, best_distance, best_weight_function, best_avg_acc = list(ranking.head(1).itertuples(index=False, name=None))[0]

In [12]:
best_knn_k, best_distance, best_weight_function

(5, 'correlation', 'uniform')

In [13]:
test(best_knn_k, best_distance, best_weight_function)

Accuracy: 0.962394303516


Accuracy: 0.966177125056
